In [23]:
import itertools
from tabulate import tabulate
import time

In [24]:
#helper function to generate the 1st Itemset with its support value.
def gen_C1(transactions):
    c1 = []
    Dict = {}
    for trans in transactions:
        for item in trans:
            if item not in Dict:
                Dict[item] = 1
            else:
                 Dict[item] = Dict[item] + 1
    for i in Dict:
        t = []
        t.append(i)
        c1.append(t)
        c1.append(Dict[i])
        t = []
    return c1

In [25]:
# helper function to generate frequent itemset i.e L(k) sets for K iteration by passing the previous candidate set C(k-1) as input to the function.
def get_frequent_itemSet(transactions,prev_c,frequent_set):
    frequent_Itemset = []
    for i in range(len(prev_c)):
        if i%2 != 0:
            support_value = (prev_c[i] * 1.0 / nums_transactions) * 100
            #support_value = (prev_c[i] / nums_transactions)
            if support_value < min_support:
                discarded_itemset.append(prev_c[i-1])
            else:
                frequent_Itemset.append(prev_c[i-1])
                frequent_Itemset.append(prev_c[i])

    for items in frequent_Itemset:
        frequent_set.append(items)

    if len(frequent_Itemset) == 2 or len(frequent_Itemset) == 0:
        return_set = frequent_set
    #if len(frequent_Itemset)<=2:
        #print("This will be returned")
        return return_set

    else:
        get_next_c(transactions, discarded_itemset, frequent_Itemset)


In [26]:
# helper function to generate Candidate sets by taking previous frequent sets as the input.
def get_next_c(transactions, prev_discarded_itemset, frequent_Itemset):
    itemset = []
    combination_set = []
    combination_set_sorted= []
    combination_set_unique = []
    current_c = []
    for i in range(len(frequent_Itemset)):
        if i%2 == 0:
            itemset.append(frequent_Itemset[i])
    for item in itemset:
        l = itemset.index(item)
        temp = []
        for i in range(l + 1, len(itemset)):
            
            for j in item:
                if j not in temp:
                    temp.append(j)
            for k in itemset[i]:
                if k not in temp:
                    temp.append(k)
                    
            combination_set.append(temp)
            #reset temp
            temp = []
            
    
    for i_set in combination_set:
        combination_set_sorted.append(sorted(i_set))
        
    #Filter out unique item set    
    for s_set in combination_set_sorted:
        if s_set not in combination_set_unique:
            combination_set_unique.append(s_set)
    combination_set = combination_set_unique
    
    for item in combination_set:
        counter = 0
        for trans in transactions:
            if set(item).issubset(set(trans)):
                counter += 1
        if counter != 0:
            current_c.append(item)
            current_c.append(counter)
    
    get_frequent_itemSet(transactions,current_c,frequent_set)

In [27]:
#helper function to generates Association Rules
def generate_rules(freq_set):
    rules = []
    for item in freq_set:
        if isinstance(item, list):
            if len(item) != 0:
                dec_length = len(item) - 1
                while dec_length > 0:
                    t = []
                    k = []
                    comb = list(itertools.combinations(item, dec_length))
                    for j in comb:
                        k = set(item) - set(j)
                        t.append(list(k))
                        t.append(list(j))
                        rules.append(t)
                        t = []
                    dec_length = dec_length - 1
    return rules

In [28]:
# helper function to displays the final output of the apriori algorithm by passing Association Rules as the input
def display_associations(rules, transactions):
    association_rules = []
    for rule in rules:
        counter_x = 0
        counter_xy = 0
        
        for trans in transactions:
            if set(rule[0]).issubset(set(trans)):
                counter_x += 1
            if set(rule[0] + rule[1]).issubset(set(trans)):
                counter_xy += 1
         
        
        #calculate support: support(x)=count(x) in transaction/total number of transactions
        support_x = 0
        support_x = (counter_x * 1.0 / nums_transactions) * 100
        
        support_xy = 0
        support_xy = (counter_xy * 1.0 / nums_transactions) * 100
        
        
        #calculate confidence: confidence(x->y) = support(x->y)/support(x)
        confidence_xy = (support_xy / support_x) * 100
        
        
        if confidence_xy >= min_confidence:
            #supportOfXAppendString = "Support Of X: " + str(round(supportOfXinPercentage, 2))
            support_xandy = "Support of X -> Y: " + str(round(support_xy))
            confidence_xandy = "Confidence: " + str(round(confidence_xy))

            #association_rules.append(supportOfXAppendString)
            association_rules.append(rule)
            association_rules.append(support_xandy)
            association_rules.append(confidence_xandy)

    return association_rules

In [38]:
#Initialization & data loading


filename = "/Users/komaluntwal/Desktop/apriori_implementation/dataset1.txt"
print('--- Dataset 1 ----')

# User Defined minimum support & minimum confidence value
min_Support = input('Enter minimum Support percentile: ')
min_support = int(min_Support)

min_Confidence = input('Enter minimum Confidence percentile: ')
min_confidence = int(min_Confidence)
print("\n") 

## ------------------------------------- # execution time start -----------
Start_Time = time.time()
##-------------------------------------------------------------------------

# Loading Transactions
transactions=[]
with open(filename,'r') as fp:
    lines = fp.readlines()
for trans in lines:
    trans = trans.rstrip()
    transactions.append(trans.split(","))

nums_transactions = len(transactions)

#temporary storage
discarded_itemset = []
frequent_set = []

#generate c1: first candidate itemset
C1 = gen_C1(transactions)
print('Candidate Set (C1): ')
candidate_set_c1 = []
for i in range(len(C1)):
    if i%2 == 0:
        candidate_set_c1.append(C1[i])
print(candidate_set_c1)        
        

#Generate subsequent L & C sets orf frequent itemset based on previously obtained frequent set
frequent_ItemSet = get_frequent_itemSet(transactions,C1,frequent_set)
#print('frequent_ItemSet: ', frequent_ItemSet)

#Association Rules:
Rules = generate_rules(frequent_set)
Association_Rules = display_associations(Rules, transactions)


#for rule in Association_Rules:
#    print(rule)

print('\n Association Rules: ' )
inc_ = 3
for rule in Association_Rules:
    if inc_ == 3 :
        print('\n')
        print(str(rule[0]) + "------>" + str(rule[1]),end='  ')
        inc_ = 0
    else:
        print(rule, end='  ')
    inc_ += 1
    
## ------------------------------------- # execution time start -----------
Execution_Time = time.time() - Start_Time
##-------------------------------------------------------------------------

print('\n\n Execution_Time: ', Execution_Time)

--- Dataset 1 ----
Enter minimum Support percentile: 10
Enter minimum Confidence percentile: 10


Candidate Set (C1): 
[['wine'], ['clocks'], ['donuts'], ['Wall prints'], ['cooking oils'], ['spices'], ['tea'], ['milk'], ['printers'], ['MP3 players'], ['paper shredders'], ['laptops'], ['Vases'], ['Novelty Makeup'], ['coffee'], ['cell phones'], ['Removals'], ['flatscreen TVs'], ['television sets'], ['DVD players'], ['Perfumes'], ['sugar'], ['Candles'], ['Table lamps'], ['Rugs'], ['Dyes']]

 Association Rules: 


['wine']------>['clocks']  Support of X -> Y: 10  Confidence: 40  

['clocks']------>['wine']  Support of X -> Y: 10  Confidence: 50  

['wine']------>['donuts']  Support of X -> Y: 10  Confidence: 40  

['donuts']------>['wine']  Support of X -> Y: 10  Confidence: 67  

['wine']------>['television sets']  Support of X -> Y: 10  Confidence: 40  

['television sets']------>['wine']  Support of X -> Y: 10  Confidence: 50  

['donuts']------>['clocks']  Support of X -> Y: 10  Confid